### IMPORTS

In [ ]:
import os
import numpy as np
import cv2
import pandas as pd
import tensorflow as tf
from tqdm import tqdm
from tensorflow.keras import layers, models
from sklearn.utils import shuffle
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

### SETTING THE DATASET PATH

In [ ]:
dataset_path = os.listdir("D://signatures/")
signature_types = os.listdir("D://signatures/")
print(signature_types)

In [ ]:
print("Types of signature: ", len(dataset_path))

In [ ]:
signs = []
for item in signature_types:
  all_signs = os.listdir("D://signatures/"+"/"+item)
  for sign in all_signs:
    if sign.endswith(".png"): 
      signs.append((item, str("D://signatures/"+"/"+item) + "/" + sign))
print(signs)

In [ ]:
signs_df = pd.DataFrame(data = signs, columns = ["sign type", "image"])
print(signs_df.head())

In [ ]:
print("Total number of signs in the dataset: ", len(signs_df))

In [ ]:
sign_count = signs_df["sign type"].value_counts()
print("Signatures in each category:")
print(sign_count)

### PRE-PROCESSING: RESIZING & DENOISING

In [ ]:

path = "D://signatures/"
image_size = 227
images = []
labels = []

for i in signature_types:
  data_path = path + str(i)
  filenames = [i for i in os.listdir(data_path)]
  for f in tqdm(filenames, desc=f"Processing {i}"):
    if f.endswith(".png"):
      img = cv2.imread(data_path + "/" + f)
      img = cv2.resize(img,(image_size, image_size))
      denoise_img = cv2.fastNlMeansDenoisingColored(img,None,2,5,9,23)
      images.append(denoise_img)
      labels.append(i)

In [ ]:
print(labels)

In [ ]:
images = np.array(images)
images.shape

In [ ]:
images = images.astype('float32') / 255.0

In [ ]:
y = signs_df['sign type'].values
y = y.reshape(-1,1)
columnTransformer = ColumnTransformer([('encoder', OneHotEncoder(), [0])], remainder='passthrough')
Y=np.array(columnTransformer.fit_transform(y),dtype='float32')
print(Y.shape)

### SPLITTING THE DATASET (60 : 40)

In [ ]:
images, Y = shuffle(images,Y,random_state=1)
train_x,test_x,train_y,test_y = train_test_split(images, Y, test_size=0.4, random_state=415)
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
train_x = train_x / 255.0
test_x = test_x / 255.0

### PROPOSED CNN MODEL

In [ ]:
cnn = models.Sequential([
     layers.Conv2D(filters=64, kernel_size=(3, 3), activation= 'relu', input_shape=(227, 227, 3)),
     layers.MaxPooling2D((2, 2)),
     layers.Conv2D(filters=128, kernel_size=(3, 3), activation= 'relu'),
     layers.MaxPooling2D((2, 2)),
     layers.Conv2D(filters=256, kernel_size=(3, 3), activation= 'relu'),
     layers.MaxPooling2D((2, 2)),
     layers.Flatten(),
     layers.Dense(256, activation= 'relu'),
     layers.Dense(2, activation= 'sigmoid')
 ])


In [ ]:
loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True)
cnn.compile(optimizer='adam',loss=loss_function,metrics=['accuracy'])

In [ ]:
# train_y = train_y.astype('float32')
cnn.fit(train_x, train_y, epochs = 5)

In [ ]:
# test_y = test_y.astype('float32')
cnn.evaluate(test_x, test_y)

In [ ]:
pred_y = cnn.predict(test_x)
pred_y[:5]

In [ ]:
classes_y = [np.argmax(element) for element in pred_y]
classes_y[:5]

In [ ]:
test_y[:5]